In [1]:
NAME = "NS_2d_tf2"

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("../../")
from Seismic_wave_inversion_PINN.tf_model_utils import *
from Seismic_wave_inversion_PINN.data_utils import *

plt.rcParams.update(plt.rcParamsDefault)
tf.keras.backend.set_floatx("float32")

%load_ext autoreload
%autoreload 2

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def tanh_model(layers, adaptive = None, bias = None):
	class adaptive_dense(keras.layers.Layer):
		def __init__(self, units, input_dim):
			super(adaptive_dense, self).__init__()
			w_init = keras.initializers.GlorotUniform()
			self.w = tf.Variable(initial_value = w_init(shape = (input_dim, units), dtype = "float32"), trainable = True)
			b_init = tf.zeros_initializer()
			self.b = tf.Variable(initial_value = b_init(shape=(units,), dtype="float32"), trainable=True)
			self.a = tf.Variable(1.0, dtype = "float32", trainable = True)
			
		def call(self, inputs):
			return tf.tanh(self.a*(tf.matmul(inputs, self.w) + self.b))
	
	with tf.device("/device:GPU:0"):
		model = keras.models.Sequential()
		if adaptive == "first" or adaptive == "all":
			model.add(adaptive_dense(layers[1], layers[0]))
		else:
			model.add(keras.layers.Dense(layers[1], input_shape = (layers[0], ), activation = "tanh"))
		for i in range(1, len(layers)-2):
			if adaptive == "all":
				model.add(adaptive_dense(layers[i+1], layers[i]))
			else:
				model.add(keras.layers.Dense(layers[i+1], input_shape = (layers[i], ), activation = "tanh"))
		if bias:
			model.add(keras.layers.Dense(layers[-1], bias_initializer = keras.initializers.Constant(bias)))
		else:
			model.add(keras.layers.Dense(layers[-1], name = "last", use_bias = False))
	return model

class constant_model(keras.Model):
	def __init__(self, values):
		super(constant_model, self).__init__()
		self.values = [tf.Variable(values[0], dtype = tf.float32, trainable = True, name = "lambda_1"),
					  tf.Variable(values[1], dtype = tf.float32, trainable = True, name = "lambda_2")]
	
	def call(self, inputs):
		return self.values

layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 2]
direct_model = tanh_model(layers, adaptive = False)
inverse_model = constant_model([0.0, 0.0])

In [4]:
loss_func = keras.losses.MeanSquaredError()

def get_residue_loss(tape, out, x, y, t, lambda_1, lambda_2):
	psi, p = out[:, 0:1], out[:, 1:2]
	u = tape.gradient(psi, y)
	v = -tape.gradient(psi, x)
	u_t = tape.gradient(u, t)
	u_x = tape.gradient(u, x)
	u_y = tape.gradient(u, y)
	u_xx = tape.gradient(u_x, x)
	u_yy = tape.gradient(u_y, y)
	v_t = tape.gradient(v, t)
	v_x = tape.gradient(v, x)
	v_y = tape.gradient(v, y)
	v_xx = tape.gradient(v_x, x)
	v_yy = tape.gradient(v_y, y)
	p_x = tape.gradient(p, x)
	p_y = tape.gradient(p, y)
	eq1 = loss_func(u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy), 0.0)
	eq2 = loss_func(v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy), 0.0)
	return eq1 + eq2, u, v
		
def step(x_d, y_d, t_d, u_d, v_d):
	with tf.GradientTape(persistent = True) as tape:
		tape.watch([x_d, y_d, t_d])
		lambda_1, lambda_2 = inverse_model(0.0)
		out_d = direct_model(tf.concat([x_d, y_d, t_d], axis = 1))
		loss_dr, u_d_pred, v_d_pred = get_residue_loss(tape, out_d, x_d, y_d, t_d, lambda_1, lambda_2)
		loss_dv = loss_func(u_d_pred, u_d) + loss_func(v_d_pred, v_d)
		loss = w_dr*loss_dr + w_dv*loss_dv
	grads = tape.gradient(loss, direct_model.trainable_variables + inverse_model.trainable_variables)
	optimizer.apply_gradients(zip(grads, direct_model.trainable_variables + inverse_model.trainable_variables))
	del tape
	return loss, loss_dr, loss_dv

def train(iterations, start_iteration, datasets, batch_sizes, print_every = 100, save_every = 10000, save_path = None):
	shuffled = lambda datasets, batch_sizes: [tf.data.Dataset.from_tensor_slices(tuple(d)).shuffle(buffer_size = d[0].shape[0], \
                                                        reshuffle_each_iteration = True).repeat().batch(batch_size) \
                                        for (d, batch_size) in zip(datasets, batch_sizes)]
	
	data_s = shuffled(datasets, batch_sizes)
	iteration = start_iteration
	
	for d in zip(*data_s):
		iteration += 1 
		loss = step(*d[0])

		if iteration % print_every == 0:
			print("{}, Epoch: {}, Loss: {:.4e}, dr: {:.4e}, dv: {:.4e}".format(get_time(), iteration, *list(loss)))
			print(inverse_model.values)
		if iteration % save_every == 0:
			direct_model.save("models/{}/{}/checkpoint_{}/direct_model".format(NAME, save_path, iteration))
			inverse_model.save("models/{}/{}/checkpoint_{}/inverse_model".format(NAME, save_path, iteration))
		if iteration == start_iteration + iterations:
			break

In [5]:
from scipy.io import loadmat
N_train = 5000

data = loadmat('Data/cylinder_nektar_wake.mat')
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2
N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

dataset = tuple(map(lambda x: tf.constant(x, dtype = tf.float32), (x_train, y_train, t_train, u_train, v_train)))

In [ ]:
optimizer = keras.optimizers.Adam()
w_dr = 1.0
w_dv = 1.0

train(iterations = 200000, start_iteration = 0, 
	 datasets = [dataset], batch_sizes = [5000],
	 print_every = 500, save_every = 5000)

2020/07/14, 00:49:14, Epoch: 500, Loss: 6.6127e-02, dr: 4.8885e-03, dv: 6.1239e-02
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.63905585>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.013678687>])
2020/07/14, 00:52:12, Epoch: 1000, Loss: 5.0074e-02, dr: 3.8726e-03, dv: 4.6201e-02
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.8529369>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.015766023>])
2020/07/14, 00:55:10, Epoch: 1500, Loss: 2.6504e-02, dr: 4.1730e-03, dv: 2.2331e-02
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.8820948>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.012217662>])
2020/07/14, 00:58:09, Epoch: 2000, Loss: 1.6813e-02, dr: 3.6168e-03, dv: 1.3196e-02
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.92322505>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.011536313>])
2020/07/14, 01:01:06, Epoch: 2500, Loss: 1.0670e-02, dr

INFO:tensorflow:Assets written to: models/NS_2d_tf2/None/checkpoint_5000/direct_model/assets
INFO:tensorflow:Assets written to: models/NS_2d_tf2/None/checkpoint_5000/inverse_model/assets
2020/07/14, 01:18:59, Epoch: 5500, Loss: 4.1967e-03, dr: 1.6132e-03, dv: 2.5836e-03
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9701195>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.01255263>])
2020/07/14, 01:21:59, Epoch: 6000, Loss: 3.7291e-03, dr: 1.4560e-03, dv: 2.2731e-03
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9727092>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.012314094>])
2020/07/14, 01:24:59, Epoch: 6500, Loss: 3.2448e-03, dr: 1.3095e-03, dv: 1.9353e-03
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9752659>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.012028999>])
2020/07/14, 01:27:57, Epoch: 7000, Loss: 2.9319e-03, dr: 1.2086e-03, dv: 1.7232e-03
ListWrapper([<tf.Varia

2020/07/14, 02:54:32, Epoch: 21500, Loss: 9.5454e-04, dr: 4.7411e-04, dv: 4.8043e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99091315>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010904672>])
2020/07/14, 02:57:31, Epoch: 22000, Loss: 9.3516e-04, dr: 4.6462e-04, dv: 4.7054e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99098206>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010865367>])
2020/07/14, 03:00:30, Epoch: 22500, Loss: 9.2768e-04, dr: 4.5747e-04, dv: 4.7022e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99139315>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010903007>])
2020/07/14, 03:03:29, Epoch: 23000, Loss: 1.0694e-03, dr: 4.8461e-04, dv: 5.8484e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99155486>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010940904>])
2020/07/14, 03:06:31, Epoch: 23500, Loss: 8.6058

2020/07/14, 04:33:09, Epoch: 38000, Loss: 7.8892e-04, dr: 3.4356e-04, dv: 4.4536e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99479544>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010978176>])
2020/07/14, 04:36:09, Epoch: 38500, Loss: 5.9739e-04, dr: 3.1620e-04, dv: 2.8119e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99451774>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010940289>])
2020/07/14, 04:39:08, Epoch: 39000, Loss: 6.6170e-04, dr: 3.2433e-04, dv: 3.3737e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9946156>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010974358>])
2020/07/14, 04:42:08, Epoch: 39500, Loss: 5.7406e-04, dr: 3.0653e-04, dv: 2.6753e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99492085>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010942467>])
2020/07/14, 04:45:07, Epoch: 40000, Loss: 5.6533e

2020/07/14, 06:11:33, Epoch: 54500, Loss: 4.4410e-04, dr: 2.4428e-04, dv: 1.9982e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99641097>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010832281>])
2020/07/14, 06:14:31, Epoch: 55000, Loss: 5.1755e-04, dr: 2.5804e-04, dv: 2.5951e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9963192>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010854365>])
INFO:tensorflow:Assets written to: models/NS_2d_tf2/None/checkpoint_55000/direct_model/assets
INFO:tensorflow:Assets written to: models/NS_2d_tf2/None/checkpoint_55000/inverse_model/assets
2020/07/14, 06:17:29, Epoch: 55500, Loss: 5.1921e-04, dr: 2.6012e-04, dv: 2.5909e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99637794>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010834695>])
2020/07/14, 06:20:28, Epoch: 56000, Loss: 5.2040e-04, dr: 2.5517e-04, dv: 2.6522e-04
ListWrapper([

INFO:tensorflow:Assets written to: models/NS_2d_tf2/None/checkpoint_70000/inverse_model/assets
2020/07/14, 07:46:58, Epoch: 70500, Loss: 3.6285e-04, dr: 2.0274e-04, dv: 1.6012e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9973648>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010711237>])
2020/07/14, 07:49:56, Epoch: 71000, Loss: 4.4666e-04, dr: 2.1443e-04, dv: 2.3222e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.997244>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010753095>])
2020/07/14, 07:52:57, Epoch: 71500, Loss: 5.7645e-04, dr: 2.3666e-04, dv: 3.3979e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99746364>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010674181>])
2020/07/14, 07:55:55, Epoch: 72000, Loss: 3.9061e-04, dr: 2.0705e-04, dv: 1.8356e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9973092>, <tf.Variable 'lambda_2:0' sh

2020/07/14, 09:22:25, Epoch: 86500, Loss: 3.6867e-04, dr: 1.8579e-04, dv: 1.8288e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99782676>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010745331>])
2020/07/14, 09:25:24, Epoch: 87000, Loss: 4.9982e-04, dr: 1.9422e-04, dv: 3.0560e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99799716>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010729749>])
2020/07/14, 09:28:25, Epoch: 87500, Loss: 3.0809e-04, dr: 1.7276e-04, dv: 1.3534e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99794203>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010698539>])
2020/07/14, 09:31:24, Epoch: 88000, Loss: 3.1982e-04, dr: 1.7401e-04, dv: 1.4581e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9979875>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010701407>])
2020/07/14, 09:34:22, Epoch: 88500, Loss: 3.8124e

2020/07/14, 11:00:47, Epoch: 103000, Loss: 2.5786e-04, dr: 1.4893e-04, dv: 1.0893e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.998472>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010724379>])
2020/07/14, 11:03:46, Epoch: 103500, Loss: 2.5791e-04, dr: 1.4855e-04, dv: 1.0936e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.9984806>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010726191>])
2020/07/14, 11:06:45, Epoch: 104000, Loss: 4.9253e-04, dr: 1.9616e-04, dv: 2.9637e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99841183>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.010668433>])
2020/07/14, 11:09:44, Epoch: 104500, Loss: 2.6704e-04, dr: 1.4850e-04, dv: 1.1854e-04
ListWrapper([<tf.Variable 'lambda_1:0' shape=() dtype=float32, numpy=0.99842674>, <tf.Variable 'lambda_2:0' shape=() dtype=float32, numpy=0.01075107>])
2020/07/14, 11:12:43, Epoch: 105000, Loss: 2.871